# **Báo cáo đồ án môn "Khoa học dữ liệu ứng dụng"**

Nhóm 7:
1. 1712770 - Trương Thị Lệ Thanh - thanhtruong1712
2. 1712771 - Bùi Thái Tấn Thành - thanhtan0501
3. 1712778 - Thống A Thảo - thaothong21

Link thùng chứa Github của nhóm: [https://github.com/thanhtruong1712/DAMH_KHDLUD]

## **Mô tả bài toán**

### Giới thiệu bài toán

* **Tên cuộc thi**: [Jigsaw Rate Severity of Toxic Comments](https://www.kaggle.com/competitions/jigsaw-toxic-severity-rating)
* **Mô tả**: Đây là cuộc thi được tổ chức bởi Jigsaw vào 09/11/2021 với 2301 nhóm tham dự. Xuất phát từ việc Jigsaw hỏi mọi người đánh giá về bình luận cá nhân khi không biết bối cảnh của bình luận đó để đánh giá bình luận nào là độc hại, không độc hại. Nhưng nhiều người có cảm giác độc hại khác nhau loại độc hại (ví dụ người cảm thấy ngôn từ thù địch độc hại hơn ngôn từ tục tĩu...). Để giải quyết nó thì Jigsaw tập hợp đánh giá của nhiều người. Nhưng các chuyên gia thì chỉ ra rằng loại bỏ các thông tin có ý nghĩa. Và rồi để đơn giản hơn, Jigsaw đã cho mọi người đánh giá trên mỗi cặp bình luận, cặp nào là độc hại hơn. Nhưng có vấn đề, khi hai tập có rõ ràng về tính độc hại thì lựa chọn của mọi người đa số sẽ chính xác. Nhưng nếu hai tập là không độc hại mọi người thường chọn ngẫu nhiên. Các cặp bình luận cũng được cho các chuyên gia đánh giá. Và ở đây, ta phải cho điểm để có thể giống với các chuyên gia ở Jigsaw nhất.
* **Input**: Tập dữ liệu khoảng 14000 bình luận được cung cấp bởi Jigsaw
* **Output**: Dự đoán điểm thể hiện mức độ độc hại tương đối của các bình luận.

### Mô tả dữ liệu:
Nhiệm vụ của nhóm là dự đoán điểm thể hiện mức độ độc hại tương đối của bình luận. Bình luận nào có mức độ độc hại cao hơn sẽ nhận được điểm số cao hơn so với bình luận có mức độ độc hại thấp hơn.
### Mô tả Files:
* **comments_to_score.csv** - Chứa Id và nội dung của các bình luận.
* **validation_data.csv** - Chứa các cặp bình luận được đánh giá là ` độc hại ít ` và `độc hại nhiều`.
* **sample_submission.csv** - File mẫu để nộp lên Kaggle
* **leaderboard.csv** - File chấm điểm của cuộc thi.
<br>

## **Giải quyết bài toán**

### 1. Ý tưởng: 
* Trong đồ án này, nhóm tìm hiểu và cài đặt lại phương pháp đứng thứ [5](https://www.kaggle.com/competitions/jigsaw-toxic-severity-rating/discussion/306390) trên private leaderboard cho bài toán Jigsaw Rate Severity of Toxic Comments.
* Tác giả đã thực hiện các bước:
  - Làm sạch nội dung các bình luận bằng cách loại bỏ tags HTML, URL links, biểu tượng cảm xúc, dấu chấm câu, ký tự đặc biệt,...
  -  Sử dụng tập dữ liệu Ruddit để huấn luyện trên 2 model với MarginRankingLoss:
    + `roberta-large` với lớp tuyến tính
    + `unitary/unbiased-toxic-roberta` với lớp tuyến tính
  - Sử dụng tập dữ liệu comments_to_score huyến luyện trên 3 mô hình khác:
   + Tfidf + RIDGE
   + unitary/unbiased-toxic-roberta with custom linear layer with MSELoss
   + unitary/unbiased-toxic-roberta with custom attention layer with MSELoss
  - Tính toán chỉ số cho các thuật toán, chọn mô hình tốt nhất để dự đoán điểm số cho bình luận




### 2. Lý do lựa chọn:

  - Tác giả viết chi tiết về solution

## Nhìn lại quá trình làm đồ án

Sau bao ngày vất vả làm đồ án thì bây giờ đã kết thúc. Bây giờ là lúc để ngồi uống coffee và tĩnh tâm nhìn lại quá trình làm.

- Mỗi thành viên: Đã gặp những khó khăn gì? (Hay mọi chuyện đều thuận lợi)
- Mỗi thành viên: Có học được gì hữu ích? (Hay không học được gì)
- Nhóm: Nếu có thêm thời gian thì sẽ làm gì?

Phần này có sao thì bạn nói vậy thôi, chứ không phải là viết
cho có, hoặc tự chế ra để nghe cho hay.

## Tài liệu tham khảo

Để hoàn thành đồ án này, nhóm bạn đã tham khảo những tài liệu nào?